In [1]:
import numpy as np
import os
import cv2
import tensorflow as tf
from PIL import Image
import glob
import pandas as pd
from tqdm import tqdm

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

N_W = 32
N_H = 32
predictpic_path = './data/new_data'   #预测图片存储的路径
logpath = './logs'   

In [3]:
# 读取保存了框架和参数的pb文件
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def,
            input_map=None,
            return_elements=None,
            name="prefix",
            op_dict=None,
            producer_op_list=None
        )
    return graph

In [4]:
def data_label():
    filelist = []
    for file in glob.glob(r'data/classification-task2/*/test.csv'):  # train/test
        df = pd.read_csv(file)
        filelist.append(df)
    df = pd.concat(filelist)
    paths = list(df['path'])
    label_list = list(df['symbol_id'])
    
    
    df.drop_duplicates(['symbol_id'], inplace=True)
    old_name = list(df['latex'])
    old_label = list(df['symbol_id'])
    index = list(range(369))
    name_dict = dict(zip(old_label, old_name))
    label_dict = dict(zip(index, old_label))
    return paths, label_list, label_dict, name_dict

In [5]:
def model_predict(filepath, sess, x, y, z):
    _, _, label_dict, name_dict = data_label()

    # input = cv2.imread(filepath)
    image = cv2.resize(filepath, (32, 32))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(filepath, (32, 32))

    image = np.expand_dims(image, axis=0)
    y_out = sess.run(y, feed_dict={
        x: image,
        z: 1
    })
    output = name_dict.get(label_dict.get(np.argmax(y_out, 1)[0]))
#     print(output)
    return output

In [6]:
#预测单张图片，并将图片名和结果输出到log1.txt文件
log1 = open('./log1.txt', 'w')
graph = load_graph(os.path.join('./logs', 'frozen_model2.pb'))
x = graph.get_tensor_by_name('prefix/Placeholder:0')
y = graph.get_tensor_by_name('prefix/output_node:0')
z = graph.get_tensor_by_name('prefix/training:0')
with tf.Session(graph=graph) as sess:
    for pic in os.listdir(predictpic_path):
        img = cv2.imread(os.path.join(predictpic_path,pic))
        output = model_predict(img, sess, x, y, z)
        print('file:{}, result:{}'.format(os.path.join(predictpic_path,pic), output))
        print('file:{}, result:{}'.format(os.path.join(predictpic_path,pic), output), file = log1)
log1.close()

Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.
file:./data/new_data/v2-00002.png, result:A
file:./data/new_data/v2-00007.png, result:A
file:./data/new_data/v2-00000.png, result:A
file:./data/new_data/v2-00003.png, result:A
file:./data/new_data/v2-00069.png, result:G
file:./data/new_data/v2-00072.png, result:G
file:./data/new_data/v2-00074.png, result:G
file:./data/new_data/v2-00070.png, result:G
file:./data/new_data/v2-00068.png, result:G
file:./data/new_data/v2-00065.png, result:G
file:./data/new_data/v2-00073.png, result:G
file:./data/new_data/v2-00066.png, result:G
file:./data/new_data/v2-00009.png, result:A
file:./data/new_data/v2-00006.png, result:A
file:./data/new_data/v2-00008.png, result:A
file:./data/new_data/v2-00071.png, result:\zeta
file:./data/new_data/v2-00004.png, result:A
file:./data/new_data/v2-00001.png, result:A
file:./data/new_data/v2-00005.png, result:A
file:./data/new_data/v2-00067.

In [7]:
def model_predict_2(filepath, sess, x, y, z):
    _, _, label_dict, name_dict = data_label()

    # input = cv2.imread(filepath)
    image = cv2.resize(filepath, (32, 32))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     image = cv2.resize(filepath, (32, 32))

    image = np.expand_dims(image, axis=0)
    y_out = sess.run(y, feed_dict={
        x: image,
        z: 1
    })
    output1 = label_dict.get(np.argmax(y_out, 1)[0])
    output2 = name_dict.get(label_dict.get(np.argmax(y_out, 1)[0]))
#     print(output)
    return output1, output2

In [8]:
#计算准确率
log2 = open('./log2.txt', 'w')
graph = load_graph(os.path.join('./logs', 'frozen_model2.pb'))
x = graph.get_tensor_by_name('prefix/Placeholder:0')
y = graph.get_tensor_by_name('prefix/output_node:0')
z = graph.get_tensor_by_name('prefix/training:0')

count = 0
bools = 0
paths, label_list, _, _ = data_label()


with tf.Session(graph=graph) as sess:
    for i in tqdm(range(len(paths))):
        pic_name = paths[i].split('..')[-1][1:]
#         print(pic_name)
#         print(os.path.join('./data',str(pic_name)))
        img = cv2.imread(os.path.join('./data',pic_name))
        
        output1, output2 = model_predict_2(img, sess, x, y, z)
#         result.append(output)
        if (output1 == label_list[i]):
            count += 1
            bools = 1
        else:
            bools = 0
        print('file:{}, result:{}, bools:{}'.format(os.path.join('./data',pic_name), output2, bools), file = log2)
log2.close()
print('acc: %.8f' % (count/len(paths)))


100%|██████████| 50528/50528 [56:40<00:00, 14.86it/s]  

acc: 0.85447673
